In [1]:
!pip install "ray[tune]"

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [2]:
import pandas as pd
import numpy as np

pd.options.display.width = 200
pd.options.display.max_columns = 20

# Import Almond.csv
almondOrig: pd.DataFrame = pd.read_csv('Almond.csv')
almondOrig.columns = ['idx', 'len', 'wid', 'thk', 'area', 'perim', 'round', 'solid', 'compact', 'aspect', 'eccent', 'extent', 'convex', 'type']

#=============== PREPROCESSING ===============#

# Remove useless/redundant cols
# almond = almond.drop(['idx', 'round', 'aspect', 'eccent'], axis=1)
almond = almondOrig.drop(['idx'], axis=1)

# Convert target type to one-hot encoding & drop
typeOneHot = pd.get_dummies(almond['type'])
typeOneHot.columns = ['mamra', 'regular', 'sanora']
almond = almond.drop('type', axis=1)

# Convert numeric cols to numeric
almond = almond.apply(pd.to_numeric, errors='coerce')

print("-"*50 + " almond " + "-"*50)
print(almond)

# Impute missing values
# almond['round'].fillna(almond['round'].mean())
# almond['aspect'].fillna(almond['aspect'].mean())
# almond['eccent'].fillna(almond['eccent'].mean())

print("-"*50 + " almond " + "-"*50)
print(almond)

# Isolate len, wid, thk cols + convert to numeric
dims: pd.DataFrame = almond[['len', 'wid', 'thk']]

# Convert to numeric
dims = dims.apply(pd.to_numeric, errors='coerce')

# Replace missing values with -1
dims = dims.fillna(-1)

# Create one-hot encoding for missing values
dimsOneHot = dims.copy()
dimsOneHot[dimsOneHot != -1] = 0
dimsOneHot[dimsOneHot == -1] = 1
dimsOneHot.columns = ['len1H', 'wid1H', 'thk1H']

print("-"*50 + " dims " + "-"*50)
print(dims.head(n=20))

print("-"*50 + " dimsOneHot " + "-"*50)
print(dimsOneHot.head(n=20))

# Sort within row + delete first col (empty after sort)
dimsSorted: pd.Series = dims.apply(np.sort, axis=1)
dims = pd.DataFrame(dimsSorted.values.tolist(), index=dimsSorted.index, columns=['null', 'wid', 'len'])
dims = dims[['wid', 'len']]

# Replace len, wid, thk cols in almond with sorted values
almond[['wid', 'len']] = dims
almond.drop('thk', axis=1, inplace=True)

# Insert one-hot encoding for missing values
almond.insert(2, 'thk1H', dimsOneHot['thk1H'])
almond.insert(2, 'wid1H', dimsOneHot['wid1H'])
almond.insert(2, 'len1H', dimsOneHot['len1H'])
almond[['len1H', 'wid1H', 'thk1H']] = dimsOneHot

# Recompute round, aspect, eccent with new len, wid
almond['round'] = 4 * almond['area'] / (np.pi * almond['len']**2)
almond['aspect'] = almond['len'] / almond['wid']
almond['eccent'] = np.sqrt(1 - (almond['wid'] / almond['len'])**2)

# Split to X, Y
# Y = almond['type']
X = almond.astype(float)
Y = typeOneHot.astype(int)

# Print fully preprocessed dataset
print("-"*50 + " Inputs (X) - fully preprocessed " + "-"*50)
print(X)
print("-"*50 + " Targets (Y) - fully preprocessed " + "-"*50)
print(Y)

-------------------------------------------------- almond --------------------------------------------------
             len         wid         thk     area       perim     round     solid   compact    aspect   eccent    extent   convex
0            NaN  227.940628  127.759132  22619.0  643.813269       NaN  0.973384  1.458265       NaN      NaN  0.681193  23237.5
1            NaN  234.188126  128.199509  23038.0  680.984841       NaN  0.957304  1.601844       NaN      NaN  0.656353  24065.5
2            NaN  229.418610  125.796547  22386.5  646.943212       NaN  0.967270  1.487772       NaN      NaN  0.683620  23144.0
3            NaN  232.763153  125.918808  22578.5  661.227483       NaN  0.965512  1.540979       NaN      NaN  0.685360  23385.0
4            NaN  230.150742  107.253448  19068.0  624.842706       NaN  0.951450  1.629395       NaN      NaN  0.714800  20041.0
...          ...         ...         ...      ...         ...       ...       ...       ...       ...      ... 

In [21]:
import matplotlib.pyplot as plt
X.hist(figsize=(16, 9))
plt.tight_layout()
plt.title("All attributes")
plt.show()

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

#----- Get compute device -----#
dev = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: \033[0;94m{dev}\033[0m")

#----- Rescale features -----#
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)

#----- Split test / training -----#
trainX, testX, trainY, testY = train_test_split(X_scaled, Y.values, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
trainX = torch.tensor(trainX, dtype=torch.float32).to(dev)
trainY = torch.tensor(trainY, dtype=torch.float32).to(dev)
testX = torch.tensor(testX, dtype=torch.float32).to(dev)
testY = torch.tensor(testY, dtype=torch.float32).to(dev)

#----- Define NN -----#
class NeuralNet(nn.Module):
    def __init__(self, inDim, outDim):
        super(NeuralNet, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(inDim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            # nn.Linear(64, 64), nn.ReLU(),
            # nn.Linear(64, 64), nn.ReLU(),
            # nn.Linear(64, 64), nn.ReLU(),
            # nn.Linear(64, 16), nn.ReLU(),
            # nn.Linear(16, 64), nn.ReLU(),
            nn.Linear(64, outDim)
        )
        # self.fc1 = nn.Linear(inDim, 64)
        # self.fc2 = nn.Linear(64, 32)
        # self.fc3 = nn.Linear(32, outDim)
        # self.relu = nn.ReLU()
    
    def forward(self, x):
        # out = self.relu(self.fc1(x))
        # out = self.relu(self.fc2(out))
        # out = self.fc3(out)
        logits = self.stack(x)
        return logits


input_dim = trainX.shape[1]
output_dim = trainY.shape[1]

model = NeuralNet(input_dim, output_dim).to(dev)

#----- Define loss + optimizer -----#
criterion = nn.MSELoss()
optimizer = optim.Rprop(model.parameters())

#----- Train -----#
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(trainX)
    loss = criterion(outputs, trainY)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [ {epoch+1}/{num_epochs}\t], Loss:\t\033[0;93m{loss.item():.4f}\033[0m')

#----- Evaluate -----#
model.eval()
with torch.no_grad():
    trainPred = model(trainX)
    trainLoss = criterion(trainPred, trainY).item()
    
    testPred = model(testX)
    testLoss = criterion(testPred, testY).item()

    print()
    print(f'Train Loss: \033[0;96m{trainLoss:.4f}\033[0m')
    print(f'Test Loss: \033[0;96m{testLoss:.4f}\033[0m')



Using device: cuda
Epoch [ 10/100	], Loss:	0.1534
Epoch [ 20/100	], Loss:	0.1287
Epoch [ 30/100	], Loss:	0.1163
Epoch [ 40/100	], Loss:	0.1062
Epoch [ 50/100	], Loss:	0.0996
Epoch [ 60/100	], Loss:	0.0944
Epoch [ 70/100	], Loss:	0.0903
Epoch [ 80/100	], Loss:	0.0873
Epoch [ 90/100	], Loss:	0.0851
Epoch [ 100/100	], Loss:	0.0832

Train Loss: 0.0830
Test Loss: 0.1113


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

#----- Get compute device -----#
dev = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: \033[0;94m{dev}\033[0m")

#----- Rescale & Encode features -----#
scaler = StandardScaler()
# x = X[['len', 'wid', 'area', 'perim', 'solid', 'compact', 'extent', 'convex']].values
x = scaler.fit_transform(X.values)

# Add back one-hot encoded missing values
# other = X[['len1H', 'wid1H', 'thk1H']]
# x = np.concatenate((x, X[['len1H', 'wid1H', 'thk1H']].values), axis=1)

# encoder = LabelEncoder()
# y = encoder.fit_transform(Y)
y = Y.values

#----- Split test / training -----#
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.2, random_state=69)

# Convert to PyTorch tensors
trainX = torch.tensor(trainX, dtype=torch.float32).to(dev)
trainY = torch.tensor(trainY, dtype=torch.float32).to(dev)
testX = torch.tensor(testX, dtype=torch.float32).to(dev)
testY = torch.tensor(testY, dtype=torch.float32).to(dev)

#----- Define NN -----#
class NeuralNet(nn.Module):
    def __init__(self, inDim, outDim, dropout = 0):
        super(NeuralNet, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(inDim, 64), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(64, 64), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(32, 64), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(64, 64), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(64, outDim)
        )
        # self.fc1 = nn.Linear(inDim, 64)
        # self.fc2 = nn.Linear(64, 32)
        # self.fc3 = nn.Linear(32, outDim)
        # self.relu = nn.ReLU()
    
    def forward(self, x):
        # out = self.relu(self.fc1(x))
        # out = self.relu(self.fc2(out))
        # out = self.fc3(out)
        logits = self.stack(x)
        return logits


input_dim = trainX.shape[1]
output_dim = trainY.shape[1]
# output_dim = len(encoder.classes_)

model = NeuralNet(input_dim, output_dim).to(dev)

#----- Define loss + optimizer -----#
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters())

#----- Train -----#
num_epochs = 2000
losses = []
accuracies = []
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(trainX)
    loss = criterion(outputs, trainY)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    # _, predicted = torch.max(outputs, 1)

    # acc = (predicted == trainY).sum().item() / trainY.size(0)
    # losses.append(loss.item())
    # accuracies.append(acc)

    if (epoch+1) % 100 == 0:
        print(f'Epoch [ {epoch+1}/{num_epochs}\t], Loss:\t\033[0;93m{loss.item():.4f}\033[0m')

#----- Evaluate -----#
from typing import Optional
def metrics(pred, targ1H, confus: Optional[str] = None):
    predLen = pred.shape[0]
    pred1H = np.zeros([predLen, 3]).astype(int)
    pred1H[np.arange(predLen), pred] = 1

    truePos = np.sum(np.logical_and(pred1H == 1, targ1H == 1))
    trueNeg = np.sum(np.logical_and(pred1H == 0, targ1H == 0))
    falsePos = np.sum(np.logical_and(pred1H == 1, targ1H == 0))
    falseNeg = np.sum(np.logical_and(pred1H == 0, targ1H == 1))

    if confus is not None:
        print()
        print(f"\033[0;94mConfusion Matrix ({confus})\033[0m:")
        print("\033[0;107m    True\tFalse\t\033[0m")
        print(f"\033[0;107mPos\033[0m {truePos}   \t{falsePos}")
        print(f"\033[0;107mNeg\033[0m {trueNeg}   \t{falseNeg}")

    precision = truePos / (truePos + falsePos)
    recall = truePos / (truePos + falseNeg)
    # accuracy = (truePos + trueNeg) / (truePos + trueNeg + falsePos + falseNeg)
    accuracy = np.sum(pred1H == targ1H) / (3 * predLen)
    f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, accuracy, f1


model.eval()
with torch.no_grad():
    trainPred = model(trainX)
    trainLoss = criterion(trainPred, trainY).item()
    
    testPred = model(testX)
    testLoss = criterion(testPred, testY).item()

    # print()
    # print(f'Train Loss: \033[0;94m{train_loss:.4f}\033[0m')
    # print(f'Test Loss: \033[0;94m{test_loss:.4f}\033[0m')

    trainPred = trainPred.cpu().numpy()
    trainTarg = trainY.cpu().numpy()

    testPred = testPred.cpu().numpy()
    testTarg = testY.cpu().numpy()

    # Convert predictions to class labels
    trainPred = np.argmax(trainPred, axis=1)
    testPred = np.argmax(testPred, axis=1)

    trainPrec, trainRec, trainAcc, trainF1 = metrics(trainPred, trainTarg, confus="Train")
    testPrec, testRec, testAcc, testF1 = metrics(testPred, testTarg, confus="Test")

    # Output metrics
    print()
    print(f"\033[0;94mTrain Metrics\033[0m:")
    # print(f"\033[0;94mPrecision:\033[0m {trainPrec:.4f}")
    # print(f"\033[0;94mRecall:\033[0m {trainRec:.4f}")
    print(f"\033[0;94mAccuracy:\033[0m {trainAcc:.4f}")
    # print(f"\033[0;94mF1 Score:\033[0m {trainF1:.4f}")

    print()
    print(f"\033[0;94mTest Metrics\033[0m:")
    # print(f"\033[0;94mPrecision:\033[0m {testPrec:.4f}")
    # print(f"\033[0;94mRecall:\033[0m {testRec:.4f}")
    print(f"\033[0;94mAccuracy:\033[0m {testAcc:.4f}")
    # print(f"\033[0;94mF1 Score:\033[0m {testF1:.4f}")

Using device: cuda
Epoch [ 100/2000	], Loss:	0.0595
Epoch [ 200/2000	], Loss:	0.0395
Epoch [ 300/2000	], Loss:	0.0335
Epoch [ 400/2000	], Loss:	0.0295
Epoch [ 500/2000	], Loss:	0.0267
Epoch [ 600/2000	], Loss:	0.0248
Epoch [ 700/2000	], Loss:	0.0234
Epoch [ 800/2000	], Loss:	0.0223
Epoch [ 900/2000	], Loss:	0.0210
Epoch [ 1000/2000	], Loss:	0.0201
Epoch [ 1100/2000	], Loss:	0.0194
Epoch [ 1200/2000	], Loss:	0.0189
Epoch [ 1300/2000	], Loss:	0.0184
Epoch [ 1400/2000	], Loss:	0.0179
Epoch [ 1500/2000	], Loss:	0.0177
Epoch [ 1600/2000	], Loss:	0.0174
Epoch [ 1700/2000	], Loss:	0.0174
Epoch [ 1800/2000	], Loss:	0.0171
Epoch [ 1900/2000	], Loss:	0.0169
Epoch [ 2000/2000	], Loss:	0.0167

Confusion Matrix (Train):
    True	False	
Pos 2193   	49
Neg 4435   	49

Confusion Matrix (Test):
    True	False	
Pos 454   	107
Neg 1015   	107

Train Metrics:
Accuracy: 0.9854

Test Metrics:
Accuracy: 0.8728
